In [4]:
!pip install -q gdown transformers torch pandas numpy

##1. Google Drive에서 모델 로드

In [5]:
import os
import zipfile
import gdown
import shutil

# https://drive.google.com/file/d/1iwRFkAnueiGc9SPCzylBbPavJZ163fSz/view?usp=drive_link
file_id = '1iwRFkAnueiGc9SPCzylBbPavJZ163fSz'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'assignment5_final_model.zip'
extract_path = './assignment5_final_model'

if os.path.exists(extract_path):
    shutil.rmtree(extract_path)

print(f"Downloading model from Google Drive (ID: {file_id})...")
gdown.download(url, output, quiet=False)

print("Unzipping model...") # 압축 해제
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# 압축 푼 폴더 내 확인
inner_files = os.listdir(extract_path)

# 이중 폴더가 생성되어 모델을 정상적으로 불러오지 못 해 이중 폴더 제거 코드 추가
if len(inner_files) == 1 and inner_files[0] == 'assignment5_final_model':
    nested_folder = os.path.join(extract_path, 'assignment5_final_model')
    print("이중 폴더 감지됨. 파일을 상위로 이동합니다...")

    # 내부 폴더의 모든 파일을 상위로 이동
    for file_name in os.listdir(nested_folder):
        shutil.move(os.path.join(nested_folder, file_name), extract_path)

    # 빈 내부 폴더 삭제
    os.rmdir(nested_folder)

print("모델 준비 완료. 폴더 구조가 정리되었습니다.")

if os.path.exists(output):
    os.remove(output)

Downloading...
From (original): https://drive.google.com/uc?id=1iwRFkAnueiGc9SPCzylBbPavJZ163fSz
From (redirected): https://drive.google.com/uc?id=1iwRFkAnueiGc9SPCzylBbPavJZ163fSz&confirm=t&uuid=a98499ad-cf90-4049-ae57-7f10406934f5
To: /content/assignment5_final_model.zip
100%|██████████| 405M/405M [00:05<00:00, 70.1MB/s]


Unzipping model...
이중 폴더 감지됨. 파일을 상위로 이동합니다...
모델 준비 완료. 폴더 구조가 정리되었습니다.


## 2. Inference

In [19]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 1. 모델 로드
MODEL_PATH = './assignment5_final_model'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Loading Model from {MODEL_PATH}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
model.to(device)
model.eval()
print("Model Loaded Successfully!")

# 2. 추론 함수 정의
def predict_sentiment(text):
    # 입력된 텍스트의 감성(긍정/부정)을 예측하고 확률을 반환
    # 전처리 및 토크나이징
    inputs = tokenizer(
        text,
        return_tensors="pt",
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    # GPU로 이동
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # 추론
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # 확률 계산 (Softmax)
    probs = F.softmax(logits, dim=1).cpu().numpy()[0]

    # 결과 해석
    pred_label = probs.argmax()
    confidence = probs[pred_label] * 100

    label_map = {0: "부정 (Negative)", 1: "긍정 (Positive)"}
    result = label_map[pred_label]

    print(f"입력: \"{text}\"")
    print(f"결과: {result} (확률: {confidence:.2f}%)")
    print("-" * 50)

    return pred_label, confidence

# 3. 실제 추론 테스트
print("\n실제 추론 테스트 시작!\n")

# Case 1: 명확한 긍정
predict_sentiment("와 여기 진짜 인생 맛집입니다. 케이크가 촉촉하고 과일도 잔뜩 들었어요. 주문할 때 이것저것 여쭤봤는데 사장님도 정말 친절해요! 집에서 좀 먼데도 자주 올 것 같아요.")

# Case 2: 전반적으로는 긍정이나 살짝 불만이 섞인 리뷰
predict_sentiment("맛은 있네요. 고기가 질이 좋고 직원분이 고기도 다 구워주세요. 반찬도 다양하고 좋음. 그런데 가격이 비싸고 웨이팅도 길어서 특별한 날 말고는 안 올듯 싶네요.")

# Case 3: 전반적으로는 부정이나 살짝 칭찬이 섞인 리뷰
predict_sentiment("인테리어 깔끔하고 자리가 넓어서 분위기는 예쁨. 그런데 메인이 생각보다 별로고 가격 대비 만족도가 떨어져서 굳이 재방문은 안 할듯. 근처에 비슷한 가격대에 더 맛있는 집 많음.")

# Case 4: 명확한 부정
predict_sentiment("멀리서 일부러 찾아올 정도는 아닌 것 같네요. 리뷰 괜찮아서 온 건데 다른 데 갈 걸 후회됩니다. 시간도 돈도 아까워요.")

# Case 5: 신조어 (긍정)
predict_sentiment("가성비 오지네요. JMT!")

# Case 6: 영어 리뷰 (부정)
predict_sentiment("The restaurant looked cozy and the staff were friendly. They greeted us with a smile. But the food itself was really disappointing. The main dish was a bit cold, and the side dishes tasted like they had been sitting out for a while.")

Loading Model from ./assignment5_final_model...
Model Loaded Successfully!

실제 추론 테스트 시작!

입력: "와 여기 진짜 인생 맛집입니다. 케이크가 촉촉하고 과일도 잔뜩 들었어요. 주문할 때 이것저것 여쭤봤는데 사장님도 정말 친절해요! 집에서 좀 먼데도 자주 올 것 같아요."
결과: 긍정 (Positive) (확률: 99.93%)
--------------------------------------------------
입력: "맛은 있네요. 고기가 질이 좋고 직원분이 고기도 다 구워주세요. 반찬도 다양하고 좋음. 그런데 가격이 비싸고 웨이팅도 길어서 특별한 날 말고는 안 올듯 싶네요."
결과: 긍정 (Positive) (확률: 96.03%)
--------------------------------------------------
입력: "인테리어 깔끔하고 자리가 넓어서 분위기는 예쁨. 그런데 메인이 생각보다 별로고 가격 대비 만족도가 떨어져서 굳이 재방문은 안 할듯. 근처에 비슷한 가격대에 더 맛있는 집 많음."
결과: 부정 (Negative) (확률: 95.98%)
--------------------------------------------------
입력: "멀리서 일부러 찾아올 정도는 아닌 것 같네요. 리뷰 괜찮아서 온 건데 다른 데 갈 걸 후회됩니다. 시간도 돈도 아까워요."
결과: 부정 (Negative) (확률: 99.81%)
--------------------------------------------------
입력: "가성비 오지네요. JMT!"
결과: 긍정 (Positive) (확률: 99.75%)
--------------------------------------------------
입력: "The restaurant looked cozy and the staff were friendly. They greeted us with a smile. But the food

(np.int64(1), np.float32(93.188866))